# Парсинг HTML. BeautifulSoup.

### Скрейпинг и парсинг данных одной страницы.

In [ ]:
# Импорт необходимых библиотек
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import urllib.parse

## Скрейпинг нескольких страниц

In [ ]:
name =[]
price =[]
form = []
content =[]
manufacturer =[]
output = {}

url_1 = 'https://www.aptekaru.de'
url = 'https://www.aptekaru.de/online-shop/allergie-heuschnupfen'

while True:
  
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  next_page_link = soup.find('a', ('class', 'arrow next'))
  result = soup.find_all('div', ('class','product-box-container'))

  url_2 = []
  for i in result:
    for link in i.find_all('div', ('class','product-box__image')):
      url_2.append(link.find('a').get('href'))
  
  url_joined = []

  for link in url_2:
    url_joined.append(urllib.parse.urljoin(url_1, link))
  
  for i in url_joined:
    response = requests.get(i)

    soup = BeautifulSoup(response.content, 'html.parser')

    # Парсинг названия товара. Обработка исключения: добавляем пустую строку.
    try:
      name.append(soup.find('h1', ('class', 'h2 mt-0 sm:mb-6')).text)
    except:
      name.append('')

    # Парсинг цены товара. 
    try:
      p = soup.find('p',('class','price-details text-grey-darker text-4xl font-sans font-bold mb-0')).find('span').text
      p = p.replace(',', '.')
      p = float(re.sub(r'[^\d.]+', '', p))
      price.append(p)
    except:
      price.append('')

    # Парсинг формы выпуска товара.
    try:
      form.append(soup.find('div', text=re.compile('Форма выпуска:')).get_text(strip=True)[15:])
    except:
      form.append('')

    # Парсинг содержания (веса) товара.  
    try:  
      cont = soup.find('div', text=re.compile('Содержание:')).get_text(strip=True)
      cont = int(re.findall(r'\d+', cont)[0])
      content.append(cont)
    except:
      content.append('')

    # Парсинг производителя товара.
    try:
      manufacturer.append(soup.find('div', text=re.compile('Производитель:')).get_text(strip=True)[15:])
    except:
      manufacturer.append('')

    output = {'Name' : name, 'Price' : price, 'Form' : form, 'Content' : content, 'Manufacturer' : manufacturer}

  if not next_page_link:
    break
  
  url = url_1 + next_page_link['href']



In [ ]:
df = pd.DataFrame(output)
df